In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` 
import org.apache.spark.sql._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
import org.joda.time.format.DateTimeFormat
import java.util.Properties
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.{DataFrame, SQLContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.joda.time.{DateTime, Days}
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.SaveMode._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.json4s._
import org.json4s.jackson.JsonMethods._
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
val formatter = DateTimeFormat.forPattern("yyyy-MM-dd")
import scala.util.Try
import java.sql.{Connection, DriverManager, ResultSet}
import org.apache.spark.SparkContext  
import org.apache.spark.SparkConf
import org.apache.spark.sql.Column
import org.apache.spark.sql.types.{DataType, DateType, TimestampType}
import org.apache.spark.sql.functions._
val spark = {
  SparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
def sc = spark.sparkContext
import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import $ivy.$                                   

import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

import org.joda.time.format.DateTimeFormat

import java.util.Properties

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.{DataFrame, SQLContext}

import org.apache.spark.{SparkConf, SparkContext}

import org.joda.time.{DateTime, Days}

import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.sql.SaveMode._

import org.apache.spark.sql.expressions.Window

import org.apache.spark.SparkConf

import org.apache.spark.sql.SparkSession

import org.json4s._

import org.json4s.jackson.JsonMethods._

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

formatter: org.joda.time.format.DateTimeFormatter = org.joda.time.format.DateTimeFormatter@5fc90c49
import scala.util.Try

import java.sql.{Connection, DriverManager, ResultSe

In [2]:
val frame1 = spark.read.option("header", true).option("inferSchema",true).csv("picsart_2.csv")
val df = frame1.drop("advertising_id", "timezone", "app", "version", "duration", "v")
df.printSchema()


root
 |-- device_id: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- user_id: long (nullable = true)



frame1: DataFrame = [device_id: string, country_code: string ... 12 more fields]
df: DataFrame = [device_id: string, country_code: string ... 6 more fields]

In [85]:
import scala.math

def group_count(dataframe:DataFrame, group_col:Array[String], count:Array[String]):DataFrame = {
        var count_col:Array[Column] = count.map(c => countDistinct(col(c)).as("count_"+c))
        var grouped_frame = dataframe.groupBy(group_col.head, group_col.tail:_*).agg(count_col.head, count_col.tail:_*)
        grouped_frame
    }
def DAU(dataframe:DataFrame, day:String):Int = {
        var frame = dataframe.withColumn("date", to_date(col("timestamp")))
        var date_col = frame("date")
        var device_id_col = frame("device_id")
        var dau = frame.filter(date_col===day).agg(countDistinct(device_id_col)).take(1)(0)(0).toString.toInt
        dau
    }
def avg_DAU(dataframe:DataFrame):Double = {
        var frame = dataframe.withColumn("date", to_date(col("timestamp"))).filter(col("device_id").isNotNull)
        var avg_DAU_frame = group_count(frame, Array("date"), Array("device_id")).
                            agg(round(avg(col("count_device_id"))))  
        var avg_dau = avg_DAU_frame.take(1)(0)(0).toString.toDouble
        avg_dau
    }
def MAU(dataframe:DataFrame, month_num:String):Int = {
        var frame = dataframe.withColumn("date", to_date(dataframe("timestamp"))).
                       withColumn("month", month(dataframe("timestamp"))).filter(col("device_id").isNotNull)                
        var month_col = frame("month")
        var device_id_col = frame("device_id")
        var mau = frame.filter(month_col===month_num).agg(countDistinct(device_id_col)).take(1)(0)(0).toString.toInt
        mau
    }
def avg_MAU(dataframe:DataFrame):Double = {
        var frame = dataframe.withColumn("date", to_date(dataframe("timestamp"))).
                    withColumn("month", month(dataframe("timestamp"))).filter(col("device_id").isNotNull)
        var avg_MAU_frame = group_count(frame, Array("month"), Array("device_id")).
                            agg(round(avg(col("count_device_id"))))  
        var avg_mau = avg_MAU_frame.take(1)(0)(0).toString.toDouble
        avg_mau
    }
def YAU(dataframe:DataFrame, year_num:String):Int = {
        var frame = dataframe.withColumn("date", to_date(dataframe("timestamp"))).
                    withColumn("month", month(dataframe("timestamp"))).withColumn("year", year(dataframe("timestamp")))
                    .filter(col("device_id").isNotNull)
        var year_col = frame("year")
        var device_id_col = frame("device_id")
        var yau = frame.filter(year_col===year_num).agg(countDistinct(device_id_col)).take(1)(0)(0).toString.toInt
        yau
    }
def avg_YAU(dataframe:DataFrame):Double = {
        var frame = dataframe.withColumn("date", to_date(dataframe("timestamp"))).
                    withColumn("month", month(dataframe("timestamp"))).withColumn("year", year(dataframe("timestamp")))
                    .filter(col("device_id").isNotNull)
        var avg_YAU_frame = group_count(frame, Array("year"), Array("device_id")).
                            agg(round(avg(col("count_device_id"))))  
        var avg_yau = avg_YAU_frame.take(1)(0)(0).toString.toDouble
        avg_yau
    }
def stickiness(dataframe:DataFrame):Double = {
        var frame = dataframe.withColumn("date", to_date(dataframe("timestamp"))).
                    withColumn("month", month(dataframe("timestamp"))).filter(col("device_id").isNotNull)
        var stickiness_raw = avg_DAU(frame)*100/avg_MAU(frame)       
        var stickiness = scala.math.round(stickiness_raw)
        stickiness
    } 
def engagement(dataframe:DataFrame):Double = {
        var frame = dataframe.withColumn("date", to_date(dataframe("timestamp"))).filter(col("session_id").isNotNull)
        var avg_dau = avg_DAU(frame)
        var frame_1 = group_count(frame, Array("date"), Array("session_id")).
                      agg(round(avg(col("count_session_id"))))
        var sessions_num = frame_1.take(1)(0)(0).toString.toDouble
        var avg_sessions_per_dau = scala.math.round(sessions_num/avg_dau)
        avg_sessions_per_dau
    }
def avg_session_duration(dataframe:DataFrame):Double = {
        var frame = dataframe.withColumn("date", to_date(dataframe("timestamp"))).filter(col("session_id").isNotNull)
        var session_id_col = frame("session_id")
        var time_col = frame("timestamp")
        var frame_filtered = frame.select("session_id", "timestamp").distinct()
        var new_frame = frame_filtered.groupBy(session_id_col).agg(min(time_col).as("session_start"), max(time_col).
                        as("session_end")).withColumn("duration_seconds", unix_timestamp(col("session_end"))-
                        unix_timestamp(col("session_start"))) 
        var avg_duration = new_frame.agg(round(avg(col("duration_seconds")))).take(1)(0)(0).toString.toDouble
   
       avg_duration
        
   }

def retention(dataframe:DataFrame, number_of_days:Int):Unit = {
        var frame = dataframe.withColumn("date", to_date(dataframe("timestamp"))).filter(col("device_id").isNotNull)
        var device_id_col = frame("device_id")
        var date_col = frame("date")
        var df_1 = group_count(frame, Array("date"), Array("device_id")).withColumnRenamed("date", "date1")
        var df_2 = frame.select("date", "device_id").groupBy(date_col).agg(collect_set(device_id_col).as("device_id_set"))
        var df_join = df_1.join(df_2, df_1("date1") <=> df_2("date"), "inner")
        var window_spec = Window.orderBy(date_col)
        var df_3 = df_join.withColumn("set_after_days", lead("device_id_set", number_of_days) over window_spec)
        var df_4 = df_3.withColumn("common_id_set", array_intersect(col("device_id_set"), col("set_after_days")))
        var df_5 = df_4.withColumn("count_ret_ids", size(col("common_id_set"))).withColumn("retention_rate",
                    round(col("count_ret_ids")*100/col("count_device_id"), 2)).drop("date1", "device_id_set", 
                    "set_after_days", "common_id_set")
        var avg_ret_rate = df_5.agg(round(avg(col("retention_rate")), 1).as("avg_retention_rate_over_time_period"))
        println(avg_ret_rate.show())
    }

def conversion_rate(dataframe:DataFrame, stage1:String, stage2:String):Unit = {
     var frame = dataframe.select("event_type", "session_id", "timestamp").filter(col("session_id").isNotNull)
     var event_col = frame("event_type")
     var window_spec = Window.partitionBy("session_id").orderBy("timestamp")
     var count_stage_1 = group_count(frame.filter(event_col === stage1), Array("event_type"), Array("session_id")).
                         take(1)(0)(1).toString.toDouble
     val df_event = frame.filter(event_col === stage1 || event_col === stage2).
                    withColumn("lead", lead(event_col, 1) over window_spec).filter(event_col === stage1).
                    withColumn("concat_col", concat(event_col, lit(" , "), col("lead")))
     val count_stage_2 = group_count(df_event.filter(col("concat_col") === stage1+" , "+stage2), 
                       Array("concat_col"), Array("session_id")).take(1)(0)(1).toString.toInt
     val conversion_rate = count_stage_2*100/count_stage_1
                                      
     println(count_stage_1)
     println(count_stage_2)
     println("conversion rate: "+conversion_rate)
 }

def most_popular_five_consecutive_events(dataframe:DataFrame):Unit = {
        
        var window = Window.partitionBy("session_id").orderBy("timestamp")
        var new_df = dataframe.filter(col("session_id").isNotNull).
        withColumn("event_2", lead("event_type", 1) over window).withColumn("event_3", lead("event_type", 2) over window).
        withColumn("event_4", lead("event_type", 3) over window).withColumn("event_5", lead("event_type", 4) over window).
        withColumn("5_events", concat(col("event_type"),lit(","),col("event_2"),lit(","),col("event_3"),
        lit(","),col("event_4"),lit(","),col("event_5"))).select("session_id","5_events").groupBy("5_events").
        agg(countDistinct("session_id").as("number_of_users")).orderBy(col("number_of_users").desc).
        withColumn("percent_of_all_users",round(col("number_of_users")/dataframe.agg(countDistinct("session_id")).
        take(1)(0)(0).toString.toInt*100,2)).show(7, false)
    }
def event_description(dataframe:DataFrame, new_event:String):Unit = {
        var frame = dataframe.withColumn("date", to_date(dataframe("timestamp"))).filter(col("device_id").isNotNull)
        var event_col = frame("event_type")
        var frame_1 = frame.filter(event_col === new_event)
        var frame_2 = group_count(frame_1, Array("date"), Array("device_id"))
        var description = frame_2.describe()
        var quantiles = frame_2.stat.approxQuantile("count_device_id", Array(0.25, 0.5, 0.75), 0.0)
        var Q1 = quantiles(0)
        var median = quantiles(1)
        var Q3 = quantiles(2)
        var IQR = Q3-Q1
        var lower_outlier_range = Q1 - 1.5*IQR
        var upper_outlier_range = Q3 + 1.5*IQR
        var outliers = frame_2.filter(col("count_device_id")<=lower_outlier_range || col("count_device_id")>= 
                       upper_outlier_range).withColumnRenamed("date", "outlier_date").withColumnRenamed("count_device_id",
                       "outliers")
        println(description.show())
        println("Q1: "+Q1+"\n"+"median: "+median+"\n"+"Q3: "+Q3)
        println(outliers.show(30))
        
    }
def most_popular_ten_events(dataframe:DataFrame, time_start:String, time_end:String):Unit = {
        var frame = dataframe.withColumn("date", to_date(dataframe("timestamp"))).filter(col("session_id").isNotNull)
        var date_col = frame("date")
        var frame_1 = frame.filter(date_col>=to_date(lit(time_start)) && date_col<=to_date(lit(time_end)))
        var frame_2 = group_count(frame_1, Array("event_type"), Array("session_id")).sort(desc("count_session_id")).limit(10)
        println(frame_2.show(false))
    }
def most_used_tools_combinations(dataframe:DataFrame):Unit = {
        var window = Window.partitionBy("session_id").orderBy("timestamp")
        var new_df = dataframe.filter(col("session_id").isNotNull).
        withColumn("tool_2", lead("tool", 1) over window).
        withColumn("tools_comb", concat(col("tool"),lit(","),col("tool_2"))).select("session_id","tools_comb").
        groupBy("tools_comb").agg(countDistinct("session_id").as("number_of_users")).orderBy(col("number_of_users").desc).
        withColumn("percent_of_all_users",round(col("number_of_users")/dataframe.agg(countDistinct("session_id")).
        take(1)(0)(0).toString.toInt*100,2)).show(7, false)
    }

def filter_function(frame:DataFrame, filter_columns:Array[String], filter_values:Array[String]):Unit = {
   var filter_columns_1 = filter_columns.map(c => col(c))
   var n = filter_columns.length
   var frame2 = frame
   for (i <- 0 to n-1) {
   frame2 = frame2.filter(filter_columns_1(i)===filter_values(i)) }
   println(frame2.show())
}


import scala.math


defined function group_count
defined function DAU
defined function avg_DAU
defined function MAU
defined function avg_MAU
defined function YAU
defined function avg_YAU
defined function stickiness
defined function engagement
defined function avg_session_duration
defined function retention
defined function conversion_rate
defined function most_popular_five_consecutive_events
defined function event_description
defined function most_popular_ten_events
defined function most_used_tools_combinations